In [71]:
!pip install transformers

In [72]:
import tensorflow as tf
import torch

from transformers import BertTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

import pandas as pd
import numpy as np
import random
import time
import datetime

In [73]:
train = pd.read_csv(open('accommodation_review.csv'), encoding = 'utf-8', sep=',', names = ['place_id', 'review_id', 'date', 'review_point', 'review', 'preprocessed_review'])

print(train.shape)

(17811, 6)


In [74]:
train.head()

,place_id,review_id,date,review_point,review,preprocessed_review
0,9479134,3044249,2021-02-03,1.0,"♩♬♩♬입니다.전날 4만원 받았는데, 다음날 5만원..첨에는 4만원 받은적이 없다고...",입니다 전날 만원 받았는데 다음날 만원 첨에는 만원 받은 적이 없다고 우기더니 전날...
1,15256072,876254,2013-07-12,5.0,여름 휴가때마다 여기서 보낸답니다. 가리왕산자연휴양림 입구가 바로 앞에 있어서 산책...,여름휴가 때마다 여기서 보낸답니다 가리왕산 자연휴양림 입구가 바로 앞에 있어서 산책...
2,1613530490,1586165,2020-02-29,1.0,오션뷰는 좋네요^^저녁에 근처횟집가서 저녁을먹을까 고민하다가 날씨도 쌀쌀하고 귀찮기...,오션뷰는 좋네요 저녁에 근처 횟집 가서 저녁을 먹을까 고민하다가 날씨도 쌀쌀하고 귀...
3,26404600,1864635,2017-12-21,5.0,펜션이 이렇게 좋아도 되나... 싶은... 매우 훌륭합니다.,펜션이 이렇게 좋아도 되나 싶은 매우 훌륭합니다
4,9662665,2697483,2020-11-19,3.0,방이커요,방이 커요


<br>
<br>

# **전처리 - 훈련셋**

In [75]:
# 리뷰 문장 추출
sentences = train['preprocessed_review']
sentences[:10]

0    입니다 전날 만원 받았는데 다음날 만원 첨에는 만원 받은 적이 없다고 우기더니 전날...
1    여름휴가 때마다 여기서 보낸답니다 가리왕산 자연휴양림 입구가 바로 앞에 있어서 산책...
2    오션뷰는 좋네요 저녁에 근처 횟집 가서 저녁을 먹을까 고민하다가 날씨도 쌀쌀하고 귀...
3                           펜션이 이렇게 좋아도 되나 싶은 매우 훌륭합니다
4                                                방이 커요
5    듀오 청소상태 뜨악 들어가자마자 파리들이 놀고 있고 베개 밑에 뭐가 있었는지 아침에...
6    오픈하고 얼마 지나지 않았는지 일단 깨끗해서 너무 좋았어요 인테리어도 그렇고 오랜만...
7    강릉 호텔 중 이 가격대에서 강릉역에도 접근성 좋고 강릉시내 많은 맛 집들과도 밀접...
8             작년엔 이문세 올해는 이승철 콘서트 매년 광복절마다 시원한 용평에서 힐링
9    솔 게스트하우스 양양 서핑 청 시행 청 시행 비치 서핑 다 같은 곳이에요 처음에 예...
Name: preprocessed_review, dtype: object

In [76]:
# BERT의 입력 형식에 맞게 변환
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
sentences[:10]

['[CLS] 입니다 전날 만원 받았는데 다음날 만원 첨에는 만원 받은 적이 없다고 우기더니 전날 영수증 내미니까 오늘은 특실이래요 전날 방이 더 크고 시설도 더 낳은데 만 원에 사람 만들고 이런 집에 다시는 가고 싶지 않네요 [SEP]',
 '[CLS] 여름휴가 때마다 여기서 보낸답니다 가리왕산 자연휴양림 입구가 바로 앞에 있어서 산책하기 좋고 앞에 아이들이 물놀이할 수 있는 곳도 있어서 여러모로 좋은 곳이네요 더불어 사장님의 친절함과 맛깔스러운 음식들 정선에 구경할 만한 곳도 잘 가르쳐 주시더라고요 암튼 해마다 찾아도 변함없고 답답한 도시를 벗어나서 힐링할 수 있는 멋진 곳인 것 같아요 [SEP]',
 '[CLS] 오션뷰는 좋네요 저녁에 근처 횟집 가서 저녁을 먹을까 고민하다가 날씨도 쌀쌀하고 귀찮기도 해서 층 flavor 석식 뷔페를 먹기로 결정하고 투숙객 프로 할인과 추가 시 와인 무제한 이벤트 진행 중이길래 쉽게 결정하고 맛있게 저녁식사를 하려고 하는데 도가니탕에서 검은 털이 나왔네요 직원을 불러 머리카락이 있어요 라고 말하니 확인해보겠다고 그릇을 가져간 후 도가니 담겨있던 그릇을 다시 가져와서는 하는 말 돼지털이네요 도가니에 왜 돼지털이 나오나요 돼지털이 검정 털이라고 되물으니 나중엔 소털이란다 나 참 ㅋㅋㅋ 죄송하다고 하면 끝날 일 절대 우리 실수 아니고 소털이다 그래서 그 털 어쨌냐 다시 보게 달라고 하니 음식 그릇에서 털만 빼고 다시 가져오고는 없단다 모르겠단다 계산하고 가려 하니 투숙객 프로 할인과 와인 무제한은 중복할인이라 안된다 이건 뭐 강릉에 오성급 호텔 맞나 별 하나도 아깝다 글 좀 잘 쓰시지 어디 중복할인 안된다는 글이 있나라고 물으니 계산 원한 분이 위아래로 사람을 쳐다본다 직원 교육 다 시 부탁드리고요 안내문 정확히 써주세요 한글이 어렵나 이게 오성급 호텔의 안내문인가 [SEP]',
 '[CLS] 펜션이 이렇게 좋아도 되나 싶은 매우 훌륭합니다 [SEP]',
 '[CLS] 방이 커요 [SEP]',
 '[CLS] 듀오 청소상태 뜨악

In [77]:
# 라벨 추출
labels = train['review_point'].values
labels

array([1., 5., 1., ..., 5., 5., 5.])

In [78]:
# BERT의 토크나이저로 문장을 토큰으로 분리
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased', do_lower_case=False)
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

print (sentences[0])
print (tokenized_texts[0])

[CLS] 입니다 전날 만원 받았는데 다음날 만원 첨에는 만원 받은 적이 없다고 우기더니 전날 영수증 내미니까 오늘은 특실이래요 전날 방이 더 크고 시설도 더 낳은데 만 원에 사람 만들고 이런 집에 다시는 가고 싶지 않네요 [SEP]
['[CLS]', '입', '##니다', '전', '##날', '만', '##원', '받', '##았', '##는데', '다음', '##날', '만', '##원', '첨', '##에는', '만', '##원', '받은', '적', '##이', '없다', '##고', '우', '##기', '##더', '##니', '전', '##날', '영', '##수', '##증', '내', '##미', '##니', '##까', '오', '##늘', '##은', '특', '##실', '##이', '##래', '##요', '전', '##날', '방', '##이', '더', '크', '##고', '시', '##설', '##도', '더', '낳', '##은', '##데', '만', '원', '##에', '사', '##람', '만', '##들', '##고', '이런', '집', '##에', '다시', '##는', '가', '##고', '싶', '##지', '않', '##네', '##요', '[SEP]']


In [79]:
# 입력 토큰의 최대 시퀀스 길이
MAX_LEN = 128

# 토큰을 숫자 인덱스로 변환
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

input_ids[0]

array([   101,   9645,  48345,   9665,  41919,   9248,  14279,   9322,
       119118,  41850,  52292,  41919,   9248,  14279,   9748,  15303,
         9248,  14279,  74141,   9664,  10739,  39218,  11664,   9604,
        12310,  54141,  25503,   9665,  41919,   9574,  15891, 119230,
         8996,  22458,  25503, 118671,   9580, 118762,  10892,   9891,
        31503,  10739,  37388,  48549,   9665,  41919,   9328,  10739,
         9074,   9834,  11664,   9485,  31928,  12092,   9074,   8995,
        10892,  28911,   9248,   9612,  10530,   9405,  61250,   9248,
        27023,  11664,  80956,   9711,  10530,  25805,  11018,   8843,
        11664,   9495,  12508,   9523,  77884,  48549,    102,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
            0,      0,      0,      0,      0,      0,      0,      0,
      

In [80]:
# 어텐션 마스크 초기화
attention_masks = []

# 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
# 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

print(attention_masks[0])

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


In [81]:
# 훈련셋과 검증셋으로 분리
train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(input_ids,
                                                                                    labels, 
                                                                                    random_state=2018, 
                                                                                    test_size=0.1)

# 어텐션 마스크를 훈련셋과 검증셋으로 분리
train_masks, validation_masks, _, _ = train_test_split(attention_masks, 
                                                       input_ids,
                                                       random_state=2018, 
                                                       test_size=0.1)

# 데이터를 파이토치의 텐서로 변환
train_inputs = torch.tensor(train_inputs)
train_labels = torch.tensor(train_labels)
train_masks = torch.tensor(train_masks)
validation_inputs = torch.tensor(validation_inputs)
validation_labels = torch.tensor(validation_labels)
validation_masks = torch.tensor(validation_masks)				

print(train_inputs[0])
print(train_labels[0])
print(train_masks[0])
print(validation_inputs[0])
print(validation_labels[0])
print(validation_masks[0])

tensor([   101,   9920,  59095,  12092,   8942, 118707,  35506,  25503,   9576,
        119022,  11664,   9689, 118985,  10530,   8885,  18622,  62200,   9407,
        119254,  68100,   9004,  32537,   9685,  11664,   9580,  37388,  18784,
         19105,  10530,   9670,  89523,   9253,  10530,   9113,  11018,   9461,
        118963,  11102,   8870,   8855,  77884,  48549,    102,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0, 

In [82]:
# 배치 사이즈
batch_size = 32

# 파이토치의 DataLoader로 입력, 마스크, 라벨을 묶어 데이터 설정
# 학습시 배치 사이즈 만큼 데이터를 가져옴
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

<br>
<br>

# **모델 생성**

In [83]:
# GPU 디바이스 이름 구함
device_name = tf.test.gpu_device_name()

# GPU 디바이스 이름 검사
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [84]:
# 디바이스 설정
if torch.cuda.is_available():    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('No GPU available, using the CPU instead.')

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


In [85]:
# 분류를 위한 BERT 모델 생성
model = BertForSequenceClassification.from_pretrained("bert-base-multilingual-cased", num_labels=6)
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elemen

In [86]:
# 옵티마이저 설정
optimizer = AdamW(model.parameters(),
                  lr = 2e-5, # 학습률
                  eps = 1e-8 # 0으로 나누는 것을 방지하기 위한 epsilon 값
                )

# 에폭수
epochs = 4

# 총 훈련 스텝 : 배치반복 횟수 * 에폭
total_steps = len(train_dataloader) * epochs

# 처음에 학습률을 조금씩 변화시키는 스케줄러 생성
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0,
                                            num_training_steps = total_steps)

In [92]:
print(len(train_dataloader))

501


<br>
<br>

# **모델 학습**

In [87]:
# 정확도 계산 함수
def flat_accuracy(preds, labels):
    
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()

    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [88]:
# 시간 표시 함수
def format_time(elapsed):

    # 반올림
    elapsed_rounded = int(round((elapsed)))
    
    # hh:mm:ss으로 형태 변경
    return str(datetime.timedelta(seconds=elapsed_rounded))

In [90]:
# 재현을 위해 랜덤시드 고정
seed_val = 42
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# 그래디언트 초기화
model.zero_grad()

# 에폭만큼 반복
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # 시작 시간 설정
    t0 = time.time()

    # 로스 초기화
    total_loss = 0

    # 훈련모드로 변경
    model.train()
        
    # 데이터로더에서 배치만큼 반복하여 가져옴
    for step, batch in enumerate(train_dataloader):
        # 경과 정보 표시
        if step % 500 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch

        

        # Forward 수행                
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask, 
                        labels=b_labels.long())
        
        # 로스 구함
        loss = outputs[0]

        # 총 로스 계산
        total_loss += loss.item()

        # Backward 수행으로 그래디언트 계산
        loss.backward()

        # 그래디언트 클리핑
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # 그래디언트를 통해 가중치 파라미터 업데이트
        optimizer.step()

        # 스케줄러로 학습률 감소
        scheduler.step()

        # 그래디언트 초기화
        model.zero_grad()

    # 평균 로스 계산
    avg_train_loss = total_loss / len(train_dataloader)            

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(format_time(time.time() - t0)))
        
    # ========================================
    #               Validation
    # ========================================

    print("")
    print("Running Validation...")

    #시작 시간 설정
    t0 = time.time()

    # 평가모드로 변경
    model.eval()

    # 변수 초기화
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    # 데이터로더에서 배치만큼 반복하여 가져옴
    for batch in validation_dataloader:
        # 배치를 GPU에 넣음
        batch = tuple(t.to(device) for t in batch)
        
        # 배치에서 데이터 추출
        b_input_ids, b_input_mask, b_labels = batch
        
        # 그래디언트 계산 안함
        with torch.no_grad():     
            # Forward 수행
            outputs = model(b_input_ids, 
                            token_type_ids=None, 
                            attention_mask=b_input_mask)
        
        # 로스 구함
        logits = outputs[0]

        # CPU로 데이터 이동
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        
        # 출력 로짓과 라벨을 비교하여 정확도 계산
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

print("")
print("Training complete!")


======== Epoch 1 / 4 ========
Training...
  Batch   500  of    501.    Elapsed: 0:06:10.

  Average training loss: 0.70
  Training epcoh took: 0:06:10

Running Validation...
  Accuracy: 0.76
  Validation took: 0:00:14

======== Epoch 2 / 4 ========
Training...
  Batch   500  of    501.    Elapsed: 0:06:09.

  Average training loss: 0.53
  Training epcoh took: 0:06:10

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:14

======== Epoch 3 / 4 ========
Training...
  Batch   500  of    501.    Elapsed: 0:06:09.

  Average training loss: 0.45
  Training epcoh took: 0:06:10

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:14

======== Epoch 4 / 4 ========
Training...
  Batch   500  of    501.    Elapsed: 0:06:10.

  Average training loss: 0.39
  Training epcoh took: 0:06:10

Running Validation...
  Accuracy: 0.79
  Validation took: 0:00:14

Training complete!


<br>
<br>

# **새로운 문장 테스트**

In [93]:
# 입력 데이터 변환
def convert_input_data(sentences):

    # BERT의 토크나이저로 문장을 토큰으로 분리
    tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]

    # 입력 토큰의 최대 시퀀스 길이
    MAX_LEN = 128

    # 토큰을 숫자 인덱스로 변환
    input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
    
    # 문장을 MAX_LEN 길이에 맞게 자르고, 모자란 부분을 패딩 0으로 채움
    input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

    # 어텐션 마스크 초기화
    attention_masks = []

    # 어텐션 마스크를 패딩이 아니면 1, 패딩이면 0으로 설정
    # 패딩 부분은 BERT 모델에서 어텐션을 수행하지 않아 속도 향상
    for seq in input_ids:
        seq_mask = [float(i>0) for i in seq]
        attention_masks.append(seq_mask)

    # 데이터를 파이토치의 텐서로 변환
    inputs = torch.tensor(input_ids)
    masks = torch.tensor(attention_masks)

    return inputs, masks

In [94]:
# 문장 테스트
def test_sentences(sentences):

    # 평가모드로 변경
    model.eval()

    # 문장을 입력 데이터로 변환
    inputs, masks = convert_input_data(sentences)

    # 데이터를 GPU에 넣음
    b_input_ids = inputs.to(device)
    b_input_mask = masks.to(device)
            
    # 그래디언트 계산 안함
    with torch.no_grad():     
        # Forward 수행
        outputs = model(b_input_ids, 
                        token_type_ids=None, 
                        attention_mask=b_input_mask)

    # 로스 구함
    logits = outputs[0]

    # CPU로 데이터 이동
    logits = logits.detach().cpu().numpy()

    return logits

In [106]:
logits = test_sentences(['전자레인지 없음 벌레 있음 인원 추가비에 침구류 포함이라고 했는데 침구류 인원보다 적게 제공받음 최대인 원명인 곳에 화장실 개 이런 부분은 안내되어 있던 부분도 있고 자연 속에 있는 곳이니까 침구류는 더 달라고 했으며 주셨겠죠하며 이해할 수 있었습니다 하지만 마지막 날 인원 추가 숯 그릴비를 받는 부분에서 결제를 후 결제 확인표시를 하는 것을 보고 있는데 잠시 후같이 간 일행과 통화하다 아 그 추가비 결제했다는 이야기를 들었습니다 중복으로 받은 거죠 그래서 전화를 한 후 사정을 이야기하니 계좌번호를 알려주면 입금해준다고 하여 문자로 알렸는데 거기서 오는 답 두 분께서 서로 의사소통의 문제가 있어 결제를 또 하신 거네요 네 어찌 추가 금 결제하는 것이 돈을 받는 분께서 돈을 받았고 안 받았고를 하신 후 요구하거나 안내했어야 하는 건 아닌지요 원만하게 해결돼서 됐다 싶었는데 마지막 저 문자는 도대체 너희가 서로 냈는지 모르고 돈 더 낸 건데 왜 그려냐 라는 식으로 들리는 건 뭘까요 동행과 이야기하기로 제 쪽에서 추가요금을 내기로 했던 부분인데 저희가 먼저 출발하고 조금 있다 동행분이 안내소에 방미 전달하면서 혹시나 한 번 확인 한 것뿐인데 과연 이것이 저런 식의 대답으로 돌아오는 것인지 실수로 중복으로 받은 것인데 돈을 왜 서로 낸 거 몰랐냐는 식인 건지 네가 안 냈냐 네가 냈냐 이런 걸 물으면서까지 그것을 확인했어야 했는지 또한 만약 통화하다 이 부분이 얘기 안되었으면 이 숙박업소 측은 아 손님이 잘못 낸 건데 뭐 하고 지나갔을 일인지 기분 좋아 여행하고 돌아오는 길 예상치도 못한 곳에서의 문자로 당황스러웠네요 이런 글은 여기 남기는 이유는 그 연락을 담당하신 분이 사장님이 아닌지 직원분이신지 모르지만 홈페이지에도 이런 글을 남기는 곳 없고 홈페이지 바로 전화연결하는 번호는 그 담당자분 이받았고 고민하다 이런 불쾌했던 감정을 알려야 할 것 같아 글을 남깁니다'])

print(logits)
print(np.argmax(logits))

[[-1.6671095   4.633141    0.76591396 -0.3764771  -1.5845273  -0.75470215]]
1


In [108]:
logits = test_sentences(['코로나 기간 중 여행이라 체인 호텔의 안정성을 믿고 예약을 했습니다 호텔 자체의 시설이나 체크인 객실 컨디션 등에 대해서는 큰 불만이 없었으나 체크아웃 당일 오전에 침대에서 벌레를 확인했습니다 흔한 날파리류의 벌레가 아니라 물어서 알레르기를 일으킬 수 있는 종류의 벌레였습니다 부모님께서 체크아웃을 하시는 거라 저는 예약만 리셉션 직원에게 항의했지만 돌아오는 답변은 친환경 호텔이라 어쩔 수 없다 였습니다 아무리 친환경 호텔이고 아무리 장마 기간이라지만 호텔 침구에 벌레라니요 별거 아닌 벌레라 생각하실지 몰라 사진 첨부합니다 코로나 상황에 방역을 열심히 하고 계신다는데 전화 통화 완료 방역을 열심히 하신다면 더더군다나 이런 벌레는 없어야 하는 게 맞는 것 같네요 불쾌한 상황에 대한 사과는 없고 다음 분들께도 또 친환경 호텔이라 어쩔 수 없다고 하시겠죠 앞으로는 호텔 설명 시에 친환경 호텔이라 객실에서 벌레를 발견하실 수 있습니다 그렇지만 호텔은 책임지지 않는다는 내용까지 명시되어야 할 것 같습니다 아기 데리고 가시는 분들은 특히 조심하시기 바랍니다'])

print(logits)
print(np.argmax(logits))

[[-2.4617867   2.9480758   1.894509    0.6768581  -0.67458725 -1.3741947 ]]
1


In [111]:
logits = test_sentences(['한화리조트 한화 계열은 피닉스 평창 중앙일보 계열 이랑 다른 법인이다 잘못 찾아가는 일이 없도록 하자 레드동 핑크 등은 한화리조트 평창 나머지는 피닉스 평창이다 일단 주차장이 좁다 괜히 실내주차하지 말고 아예 지하층 야외주차장이 있으니 거기하면 상대적으로 여유 있게 주차할 수 있다 블루 케니언에서 너무 떨어져 있고 곧바로 연결되지 않는다 반면 피닉스 블루종은 지하층에서 센터 플라자 블루 케니언으로 연결되는 통로가 있다 차를 몰아 블루 케니언 쪽으로 이동하는 게 낫다 걷는 거리가 상당하다 블루 케니언 옆 건물이 센터 플라자다 층에 파리바게뜨가 있는 곳 괜히 비발디파크나 롯데리조트 생각하고 피닉스 블루 등 지하로 가지 말기 바란다 객실은 오래된 느낌이 확 난다 key도 아직 철제 key를 쓴다 마룻바닥 디자인은 년 전 올드 한 감성으로 색깔이 진한 갈색이나 도배나 이불장 걸레받이는 연한 나무 무늬로 최근에 바꾼 것으로 추정되어 선명한 대비를 이룬다 이왕이면 바닥재까지 바꿨으면 깨끗한 느낌이 들었을 텐데 아직 멀쩡해서 바닥재는 안 바꿨나 보다 하지만 객실은 아주 넓다 설악 쏘라노보다 넓고 전반적으로 조용하다 설악 쏘라노는 객실 앞에 식당에서 노래 틀고 코로나 이전에는 야간에 놀이기구에서 불빛 나고 해서 시끄러워 밤에 자기가 힘들었다 다만 액티비티 차원에서는 같은 한화 계열 중에서는 워터피아랑 산책로가 있는 설악 쏘라노가 나은 것 같다 이러저러한 불편사항을 생각하면 중앙에서 마저 인수해서 부지를 합치고 공사를 해서 확장하는 편이 나아 보인다 블루 케니언 도 더 크게 만들고 이 한화리조트 평창은 산책코스가 있는 것도 아니고 달랑 개 동으로 부대시설이 너무 열악하다 이 상태로는 오래 버티기엔 힘들어 보인다 인기가 많아 보이지도 않는다 대명 홍천 비발디파크랑 비교했을 때 많이 열악하다 이점은 휘닉스파크 센터 플라자도 마찬가지 콘도는 역시 대명'])

print(logits)
print(np.argmax(logits))

[[-1.4030697   2.7196681   2.0125906   0.94277585 -0.93514234 -1.962033  ]]
1


In [109]:
logits = test_sentences(['호텔 생각하면 실망하실 거예요 괜찮은 모텔 정도로 보면 적당합니다'])

print(logits)
print(np.argmax(logits))

[[-3.3275385  -0.03545406  0.5635631   1.4716722   0.9876551   0.31368235]]
3


In [107]:
logits = test_sentences(['콘도형 호텔형 두 군데 있는데 콘도형에 묵었습니다 박 취사 유무 차이 입장료는 일반인들에게 받고 투숙객은 안 받더라고요 ㅎ 콘도형 기준으로 알려드립니다 장점 날씨 좋은 날에는 뷰가 좋음 바다가 홀수방은 부분 오션 뷰 주변에 파릇한 공원 산책할 수 있음 편의시설이 좋음 작은 마트 노래방 헬스센터 입장할 때 분위기 좋음 단점 홀수방 선택 시 밤에 밖에서 객실이 확실하게 보임 커튼 필 호텔 직원들이 잘 안 웃음 콘도형은 물 비치가 안 되어있음 밤에 벌레가 많아서 문을 닫아야 함 에어컨이 좀 약해서 하루 종일 틀어도 방 전체에 그렇게 시원하지 않음 에어컨 밑은 시원합니다 ㅋㅋ 호텔 콘도 밖에는 뭐가 없어요 호텔 안에서 할 수 있긴 하지만 비치된 물품은 드라이기 샴푸 보디워시 비누 일용 수건 에프킬라 전자레인지 전기밥솥 인 기준 식기세트 냄비 종 도마 등침대 베개는 그럭저럭 푹신합니다 룸서비스로 콤비네이션 피자와 양념치킨 주문했는데요 콤비네이션은 가격에 비해 양도 질도 부족합니다 양념치킨은 옛날 치킨 맛 개인 취향 저격 조식 현재 투숙객 이벤트 할인해서 할인 가격으론 괜찮았습니다 스카이라운지 그리고 호텔형 리조트도 구경해보고 싶었는데 짧은 시간에 구경하기가 힘들었어요 개인적으로 피곤하기도 하고 공원으로 가서 스카이워크가 있었는데요 거의 급 경사 수준의 장소에 계단이 설치되어있어서 고소공포증 있는 분은 가는 길에 포기할 것 같아요 제가 그랬어요 스카이워크는 인만 들어갈 수 있습니다 순서 지키면서 차례대로 공포증 없으신 분에게는 정말 장관인 곳 여유 있으시다면 주변 뷰도 보시고 편하게 즐기시면 좋을 것 같아요 호텔형은 어떤지 모르지만 콘도형보다 좋겠죠 주변 경관으로 인해 박 더 해보고 싶은 생각이 들었어요'])

print(logits)
print(np.argmax(logits))

[[-1.3107493  -2.3357615  -1.037825    0.88406044  2.4160337   0.33331418]]
4


In [110]:
logits = test_sentences(['흠 적을게 많아서 요약하자면 인터넷상에 있는 사진은 굉장히 허름해 보이지만 실제로 가면 불편한 것 허름한 느낌 전혀 들지 않습니다 그리고 엄 청 깨끗합니다 그리고 놀란 게 수건 직접 빨래하시는 거 같은데 냄새 하나도 안 나요 친절한 사장님 사장님 정말 친절하세요 오히려 부담될 정도로 잘 챙겨주시고 좋습니다 요즘 펜션 다른 곳 가보면 눈치 많이 보이고 지켜달라는 것 많은데 이곳은 손님을 편하게 쉬게 해야겠다는 느낌을 받았습니다 가격제가 양구에서 원래 숙박하려고 했다가 예약이 다 차서 화천까지 갔는데 평균 숙박료가 일대에 모두 만 만입니다 하지만 여긴 만 천 원 바비큐도 인 기준 만원 저렴한데 깨끗해서 좋습니다 총평 가격이 싼데 허름한 사진 때문에 고민된다면 바로 예약하세요 깨끗하고 저렴하고 친절하십니다 단 수압이 쪼금 약합니다'])

print(logits)
print(np.argmax(logits))

[[-2.0824466 -1.9720962 -2.186492  -1.5564895  1.2332951  4.799001 ]]
5
